In [1]:
from gnutools import fs
from bpd.dask import DataFrame, udf
from bpd.dask import functions as F
from gnutools.remote import gdrivezip

In [2]:
# Import a sample dataset
gdrivezip("gdrive://1y4gwaS7LjYUhwTex1-lNHJJ71nLEh3fE")
df = DataFrame({"filename": fs.listfiles("/tmp/1y4gwaS7LjYUhwTex1-lNHJJ71nLEh3fE", [".wav"])})
df.compute()      

,filename
0,/tmp/1y4gwaS7LjYUhwTex1-lNHJJ71nLEh3fE/wow/919...
1,/tmp/1y4gwaS7LjYUhwTex1-lNHJJ71nLEh3fE/wow/6a2...
2,/tmp/1y4gwaS7LjYUhwTex1-lNHJJ71nLEh3fE/wow/682...
3,/tmp/1y4gwaS7LjYUhwTex1-lNHJJ71nLEh3fE/wow/beb...
4,/tmp/1y4gwaS7LjYUhwTex1-lNHJJ71nLEh3fE/wow/d37...
...,...
145,/tmp/1y4gwaS7LjYUhwTex1-lNHJJ71nLEh3fE/left/6a...
146,/tmp/1y4gwaS7LjYUhwTex1-lNHJJ71nLEh3fE/left/e3...
147,/tmp/1y4gwaS7LjYUhwTex1-lNHJJ71nLEh3fE/left/68...
148,/tmp/1y4gwaS7LjYUhwTex1-lNHJJ71nLEh3fE/left/e7...


In [12]:
# Register a user-defined function
@udf
def word(f):
    return fs.name(fs.parent(f))

# Apply a udf function
df\
.withColumn("classe", word(F.col("filename")))\
.compute()    

,filename,classe
0,/tmp/1y4gwaS7LjYUhwTex1-lNHJJ71nLEh3fE/wow/919...,wow
1,/tmp/1y4gwaS7LjYUhwTex1-lNHJJ71nLEh3fE/wow/6a2...,wow
2,/tmp/1y4gwaS7LjYUhwTex1-lNHJJ71nLEh3fE/wow/682...,wow
3,/tmp/1y4gwaS7LjYUhwTex1-lNHJJ71nLEh3fE/wow/beb...,wow
4,/tmp/1y4gwaS7LjYUhwTex1-lNHJJ71nLEh3fE/wow/d37...,wow
...,...,...
145,/tmp/1y4gwaS7LjYUhwTex1-lNHJJ71nLEh3fE/left/6a...,left
146,/tmp/1y4gwaS7LjYUhwTex1-lNHJJ71nLEh3fE/left/e3...,left
147,/tmp/1y4gwaS7LjYUhwTex1-lNHJJ71nLEh3fE/left/68...,left
148,/tmp/1y4gwaS7LjYUhwTex1-lNHJJ71nLEh3fE/left/e7...,left


In [13]:
# You can use inline udf functions
df\
.withColumn("name", udf(fs.name)(F.col("filename")))\
.display()

,filename,name
0,/tmp/1y4gwaS7LjYUhwTex1-lNHJJ71nLEh3fE/wow/919...,919d3c0e_nohash_2
1,/tmp/1y4gwaS7LjYUhwTex1-lNHJJ71nLEh3fE/wow/6a2...,6a27a9bf_nohash_0
2,/tmp/1y4gwaS7LjYUhwTex1-lNHJJ71nLEh3fE/wow/682...,6823565f_nohash_2
3,/tmp/1y4gwaS7LjYUhwTex1-lNHJJ71nLEh3fE/wow/beb...,beb49c22_nohash_1
4,/tmp/1y4gwaS7LjYUhwTex1-lNHJJ71nLEh3fE/wow/d37...,d37e4bf1_nohash_0
...,...,...
145,/tmp/1y4gwaS7LjYUhwTex1-lNHJJ71nLEh3fE/left/6a...,6a27a9bf_nohash_0
146,/tmp/1y4gwaS7LjYUhwTex1-lNHJJ71nLEh3fE/left/e3...,e32ff49d_nohash_0
147,/tmp/1y4gwaS7LjYUhwTex1-lNHJJ71nLEh3fE/left/68...,6823565f_nohash_2
148,/tmp/1y4gwaS7LjYUhwTex1-lNHJJ71nLEh3fE/left/e7...,e77d88fc_nohash_0


In [14]:
# Retrieve the first 3 filename per classe
df\
.withColumn("classe", word(F.col("filename")))\
.aggregate("classe")\
.withColumn("filename", F.top_k(F.col("filename"), 3))\
.explode("filename")\
.compute()

,filename
classe,
wow,/tmp/1y4gwaS7LjYUhwTex1-lNHJJ71nLEh3fE/wow/919...
wow,/tmp/1y4gwaS7LjYUhwTex1-lNHJJ71nLEh3fE/wow/6a2...
wow,/tmp/1y4gwaS7LjYUhwTex1-lNHJJ71nLEh3fE/wow/682...
nine,/tmp/1y4gwaS7LjYUhwTex1-lNHJJ71nLEh3fE/nine/0f...
nine,/tmp/1y4gwaS7LjYUhwTex1-lNHJJ71nLEh3fE/nine/6a...
...,...
yes,/tmp/1y4gwaS7LjYUhwTex1-lNHJJ71nLEh3fE/yes/0a9...
yes,/tmp/1y4gwaS7LjYUhwTex1-lNHJJ71nLEh3fE/yes/0a7...
left,/tmp/1y4gwaS7LjYUhwTex1-lNHJJ71nLEh3fE/left/6a...


In [15]:
# Add the classe column to the original dataframe
df = df\
.withColumn("classe", word(F.col("filename")))

# Display the modified dataframe
df.display()

,filename,classe
0,/tmp/1y4gwaS7LjYUhwTex1-lNHJJ71nLEh3fE/wow/919...,wow
1,/tmp/1y4gwaS7LjYUhwTex1-lNHJJ71nLEh3fE/wow/6a2...,wow
2,/tmp/1y4gwaS7LjYUhwTex1-lNHJJ71nLEh3fE/wow/682...,wow
3,/tmp/1y4gwaS7LjYUhwTex1-lNHJJ71nLEh3fE/wow/beb...,wow
4,/tmp/1y4gwaS7LjYUhwTex1-lNHJJ71nLEh3fE/wow/d37...,wow
...,...,...
145,/tmp/1y4gwaS7LjYUhwTex1-lNHJJ71nLEh3fE/left/6a...,left
146,/tmp/1y4gwaS7LjYUhwTex1-lNHJJ71nLEh3fE/left/e3...,left
147,/tmp/1y4gwaS7LjYUhwTex1-lNHJJ71nLEh3fE/left/68...,left
148,/tmp/1y4gwaS7LjYUhwTex1-lNHJJ71nLEh3fE/left/e7...,left


In [16]:
# Display the dataframe
# Retrieve the first 3 filename per classe
@udf
def initial(classe):
    return classe[0]
    

_df = df\
.aggregate("classe")\
.reset_index()\
.withColumn("initial", initial(F.col("classe")))\
.select(["classe", "initial"])\
.set_index("classe")

# Display the dataframe grouped by classe
_df.compute()
    

,initial
classe,
bed,b
bird,b
cat,c
dog,d
down,d
eight,e
five,f
four,f
go,g


In [17]:
_df_initial = _df.reset_index().aggregate("initial")
_df_initial.compute()

,classe
initial,
b,"[bed, bird]"
c,[cat]
d,"[dog, down]"
e,[eight]
f,"[five, four]"
g,[go]
h,"[happy, house]"
l,[left]
m,[marvin]


In [18]:
# Join the dataframes
df\
.join(_df, on="classe").drop_column("classe")\
.join(_df_initial, on="initial")\
.display()

,filename,initial,classe
0,/tmp/1y4gwaS7LjYUhwTex1-lNHJJ71nLEh3fE/wow/919...,w,[wow]
1,/tmp/1y4gwaS7LjYUhwTex1-lNHJJ71nLEh3fE/wow/6a2...,w,[wow]
2,/tmp/1y4gwaS7LjYUhwTex1-lNHJJ71nLEh3fE/wow/682...,w,[wow]
3,/tmp/1y4gwaS7LjYUhwTex1-lNHJJ71nLEh3fE/wow/beb...,w,[wow]
4,/tmp/1y4gwaS7LjYUhwTex1-lNHJJ71nLEh3fE/wow/d37...,w,[wow]
...,...,...,...
13,/tmp/1y4gwaS7LjYUhwTex1-lNHJJ71nLEh3fE/left/6a...,l,[left]
14,/tmp/1y4gwaS7LjYUhwTex1-lNHJJ71nLEh3fE/left/e3...,l,[left]
15,/tmp/1y4gwaS7LjYUhwTex1-lNHJJ71nLEh3fE/left/68...,l,[left]
16,/tmp/1y4gwaS7LjYUhwTex1-lNHJJ71nLEh3fE/left/e7...,l,[left]
